In [1]:
# Import Local Modules

from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys ### Unused
import requests
import time
import re
import pandas as pd
import os
from datetime import date

In [2]:
# Changing path of the working directory
os.chdir("C:\learnings\Jupyter Path")   # Replace your desired path in-place of mentioned path

# Check current working directory
os.getcwd()

'C:\\learnings\\Jupyter Path'

In [4]:
# Converting Excel into Pandas Dataframe
read_excel = pd.read_excel("Test_Link_check.xlsx", sheet_name = 0 ,header = None, index_col = None)
df = pd.DataFrame(data = read_excel)


In [5]:
"""Defining the Function"""

# Creating a Headless browser
option = webdriver.ChromeOptions()
option.headless = True

def extract_product_count(url:str) -> list:
    """This function will extract product-counts from the websites mentioned in the Input Csv
   
    Argument: 
       url: str content
       
    Return:
        response: list
    
    """
    
    # Instance of webdrive (Interface of browser that enables controls of browsers)
    
    driver = webdriver.Chrome("C:\learnings\Jupyter Path\\chromedriver.exe", options= option)
    
    driver.get(url)   
        
    # Stabilize the webpage
    time.sleep(6)    
    page = driver.page_source
        
    # Exiting the webdriver
    driver.quit() 
        
    # Get the website's Static HTML DOM 
    soup = BeautifulSoup(page, 'html.parser')
        
    # Find the product count in the HTML DOM through X-path
    get_text = soup.find('span', {'class':"header-product-count"})
        
    # Check if Product count available on website
    
    if get_text is None:
        return [url, "Count Not Available on Website"]
    
    response = get_text.text
        
    # Remove parenthesis
    trimed_response = re.sub(r'[()]',"",response) 

    # Convert the count into number and trim whitespaces
    final_output = int(trimed_response.strip())

    # Return urls and product count in list format
    return [url, final_output]

    ### Unnecessary else statement. Better to write it as the following:
    """
    if get_text is None:
        return [url, "Count Not Available"]
    
    response = get_text.text
        
    # Remove parenthesis
    trimed_response = re.sub(r'[()]',"",response) 

    # Convert the count into number and trim whitespaces
    final_output = int(trimed_response.strip())

    # Return urls and product count in list format
    return [url, final_output]
    """
    ### As seen above, we take care of the exception case early on and then go on with the rest of the logic.
    ### There's no need to use else because if the condition is met, the function will return and the rest will be ignored

In [6]:
# Empty array to store data
data = []

for i in df.index:
    """Iteration through working Urls and storing product counts of each"""
    
    url = df[0][i]
    response = requests.get(url)
        
    ### In some cases the status could be 201
    # Check the status of response
    if response.status_code == 200 or 201:     
        product_count = extract_product_count(url)
            
        # Append output in list
        data.append(product_count)
            
    else:
        ### What about 50x? Needs better fault management. Maybe you can print the status code as well 
        ### For example: 'Could not extract product count. Page Status:' + response.status_code
        # If status is 404 forbidden
        print("Could not extract product count . Page Status: " + response.status.code)
        ### Absolutely unnecessary continue. The loop will continue without you telling it to
        ### If you really want to use continue, then use it inside the if condition and remove the else


C:\Users\prasad.more\AppData\Local\Temp\ipykernel_22424\3709359442.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\learnings\Jupyter Path\\chromedriver.exe", options= option)


In [7]:
# Creating a Dataframe to structure data
dataframe = pd.DataFrame(data = data, columns = ["Urls", "Product Count"])

# Convert the output to excel
dataframe.to_excel("Product_Count.xlsx", sheet_name = "Trail Run" )

"""Check a excel file named "Product_Count_2" generated in your working directory"""

'Check a excel file named "Product_Count_2" generated in your working directory'

In [ ]:
!git commi